In [ ]:
# Da qui si può selezionare la lingua per cui fare l'analisi
# lingue selezionabili: portoghese, spagnolo, tedesco, italiano, francese, inglese
lingua_selezionata = 'spagnolo'

# Topic modeling

In [ ]:
'''TOPIC MODELING PER LINGUA'''
import json
import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from gensim import corpora, models
import pyLDAvis.gensim

# seleziona la lingua in base a quella selezionata ad inizio file
with open(lingua_selezionata + '.json', 'r') as f:
    tweets_data = json.load(f)
    
# analisi della topic modelling
# seleziona la lingua in base a quella selezionata ad inizio file
match lingua_selezionata:
    case 'portoghese':
        selected_language = 'portuguese'
        stopwords = ['nuclear','https','http','co','amp','rt']
    case 'spagnolo':
        selected_language = 'spanish'
        stopwords = ['nucleares','nuclear','https','http','co','amp','rt']
    case 'tedesco':
        selected_language = 'german'
        stopwords = ['nuklearia','https','http','co','amp','rt']
    case 'italiano':
        selected_language = 'italian'
        stopwords = ['nucleare','https','http','co','amp','rt','poi','fa','della','delle','dell','essere','può','senza','solo','anche','così','quindi','dopo','fatto','fino','sì','cosa','parte','no','due','già','ora','nessuna','invece','tre','quale','quando','prima','tutto','stati','stata']
    case 'francese':
        selected_language = 'french'
        stopwords = ['ça','a','nucléaire','nucléaires','https','http','co','amp','rt']
    case 'inglese':
        selected_language = 'english'
        # stopwords che abbiamo deciso di non considerare nono presenti nella lista dalla libreria nltk
        stopwords = ['https','http','co','it','would','the','amp','use','go','one','name','get','rt','nuclear']

# Costruzione del corpus
corpus = []
for item in tweets_data:
    # Tokenization del testo
    parole = word_tokenize(item['text'].lower())
    # rimuovi i simboli di punteggiatura o non parole e le stopwords
    parole = [parola for parola in parole if parola.isalpha()]
    parole = [parola for parola in parole if parola not in nltk.corpus.stopwords.words(selected_language)]
    parole = [parola for parola in parole if parola not in stopwords]
    corpus.append(parole)

# Costruzione del dizionario
dizionario = corpora.Dictionary(corpus)

# Costruzione della matrice dei documenti
doc_term_matrix = [dizionario.doc2bow(doc) for doc in corpus]

# Costruzione del modello LDA
lda_model = models.LdaMulticore(corpus = doc_term_matrix, num_topics=5, id2word=dizionario)

# Visualizzazione dei topic
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
# stampa grafica dei print_topic


In [ ]:
import os
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(5))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dizionario)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(5) +'.html')
LDAvis_prepared